In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# specify substep parameters for interactive run
# this cell will be replaced during job run with the parameters from json within params subfolder
substep_params={
    "mae_threshold":1.5 ,
    "mape_threshold":0.8}

In [3]:
# load pipeline and step parameters - do not edit
from sinara.substep import get_pipeline_params, get_step_params
pipeline_params = get_pipeline_params(pprint=True)
step_params = get_step_params(pprint=True)

**Pipeline params:**


{'X': 'something',
 'env_name': 'user',
 'pipeline_name': 'pipeline',
 'zone_name': 'zone'}




**Step params:**


{'Y': 'something_else'}




In [4]:
#3 define substep interface
from sinara.substep import NotebookSubstep, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params)

substep.interface(
   
    inputs =
    [
        { STEP_NAME: "model_train", ENTITY_NAME: "california_bento" }
    ],
    outputs = 
    [
    ]
    
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

**STEP NAME:**


'model_test'




**INPUTS:**


[{'user.pipeline.zone.model_train.california_bento': '/data/home/jovyan/pipeline/zone/model_train/run-25-01-15-070811/california_bento'}]




In [5]:
#4 run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

Session is run


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/15 08:55:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/15 08:55:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:

# read inputs 
bento_step_inputs = substep.inputs(step_name="model_train")

In [10]:
#6 make something to create artifacts
# Test model API
from sinara.bentoml import start_dev_bentoservice, stop_dev_bentoservice, load_bentoservice
from sklearn.metrics import mean_absolute_percentage_error,mean_absolute_error
import pandas as pd
import requests
import json

# Load BentoService
bento_serv = load_bentoservice(bento_step_inputs.california_bento)

# Stop a dev model server if running
stop_dev_bentoservice(bento_serv)

# Start a dev model server to test out the API endpoint locally
start_dev_bentoservice(bento_serv)

serv_v = json.loads(requests.post("http://127.0.0.1:5000/service_version", json={}).text)
## убрал вывод неудобно
#print(serv_v)

test_data = json.loads(requests.post("http://127.0.0.1:5000/test_data", json={}).text)

preds = json.loads(requests.post("http://127.0.0.1:5000/predict", 
                                   json=test_data['X']).text)

mae = mean_absolute_error(pd.DataFrame(test_data['Y']).values, preds)
mape = mean_absolute_percentage_error(pd.DataFrame(test_data['Y']).values, preds)

print("The mean_absolute_error (MAE) on test set: {:.4f}".format(mae))
print("The mean_absolute_percentage_errorr (MAPE) on test set: {:.4f}".format(mape))

# Stop the dev model server
stop_dev_bentoservice(bento_serv)

[2025-01-15 08:59:50,650] WARNING - Python 3.10.12 found in current environment is not officially supported by BentoML. The docker base image used is'bentoml/model-server:0.13.2' which will use conda to install Python 3.10.12 in the build process. Supported Python versions are: f3.6, 3.7, 3.8
[2025-01-15 08:59:50,773] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.13.2, but loading from BentoML version 0.13.2+6.g2dc5913c
[2025-01-15 08:59:50,776] WARNING - Module `model_service` already loaded, using existing imported module.
[2025-01-15 08:59:50,793] WARNING - No dev server is running.
[2025-01-15 08:59:51,791] INFO - BentoService bundle 'ModelService:user.pipeline.zone.california_bento.run-25-01-15-070811' created at: /tmp/tmpf016k5wp
[2025-01-15 08:59:51,798] INFO - ======= starting dev server on port: 5000 =======
[2025-01-15 08:59:52,568] WARNING - Using BentoML not from official PyPI release. In order to find the sa

In [11]:
# check eval result
mae_threshold = substep_params["mae_threshold"]
mape_threshold = substep_params["mape_threshold"]
if mae > mae_threshold:
    raise Exception(f'MAE is {mae}, more than acceptable value of {mae_threshold}')
if mape > mape_threshold:
    raise Exception(f'MAPE is {mape}, more than acceptable value of {mape_threshold}')

[2025-01-15 09:00:07,190] WARNING - Python 3.10.12 found in current environment is not officially supported by BentoML. The docker base image used is'bentoml/model-server:0.13.2' which will use conda to install Python 3.10.12 in the build process. Supported Python versions are: f3.6, 3.7, 3.8


In [12]:
# stop spark
SinaraSpark.stop_session()